# UNET Model for Classification and Segmentation

In [1]:
from importsfunctions import *

## Model Creation e load de pesos:

In [ ]:
model = redeUNETconjunta(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS, act='relu',complexity=32,fully=1024,pool=3)
#model.summary()

### Bloquear pesos da parte de treino:

In [ ]:
for layer in model.layers:
    layer.trainable= False
    #print(layer.name)
    if layer.name in ['classification_output','dense','max_pooling2d_4']:
        layer.trainable = True

In [ ]:
for layer in model.layers:
    layer.trainable= False
    #print(layer.name)
    if layer.name in ['segmentation_output','conv2d_18','conv2d_17','conv2d_16',
 'conv2d_transpose_3','conv2d_15','conv2d_14','conv2d_transpose_2','conv2d_13','conv2d_12','conv2d_11','conv2d_transpose_1','conv2d_10']:
        layer.trainable = True

### Compile e summary

In [ ]:
alfa=0.8
model.summary()
model.compile(optimizer=Adam(learning_rate=1e-4, ),
              loss={'classification_output': 'categorical_crossentropy', 'segmentation_output': 'binary_crossentropy'},
              loss_weights={'classification_output':(1- alfa), 'segmentation_output': (alfa)},#2
              metrics={'classification_output': 'categorical_accuracy', 'segmentation_output':[jaccard,dice] },#[jaccard,dice]
              weighted_metrics={'classification_output': 'accuracy'},
             sample_weight_mode={'classification_output': None, 'segmentation_output': 'temporal'})

## Configuration

In [ ]:
['segmentation_output','conv2d_37','conv2d_36','conv2d_35',
 'conv2d_transpose_7','conv2d_34','conv2d_33','conv2d_transpose_6','conv2d_31','conv2d_32','conv2d_30','conv2d_transpose_5','conv2d_29']
# Parameters
batch=8
paramsT = {'batch_size': batch,
          'n_classes': 3,
          'shuffle': True,
         'validation': False}
paramsV = {'batch_size': 2,
          'n_classes': 3,
          'shuffle': False,
         'validation': True}

# Datasets
DATAPATH_csv='ISIC-2017_Training_Part3_GroundTruth_r.csv'
DATAPATH_segm_csv='ISIC-2017_Validation_Part3_GroundTruth.csv'
partition,labels = create_dictionary(DATAPATH_csv,DATAPATH_segm_csv)

# Generators
training_generator = DataGenerator(partition['train'], labels, **paramsT)
validation_generator = DataGenerator(partition['validation'], labels, **paramsV)

### CallBacks

In [ ]:
Reduce=ReduceLROnPlateau(factor=0.7, patience=10, min_lr=1e-9, verbose=1)

#filepath='Weights/ClassSegm/final.h5'
#filepath1='Weights/ClassSegm/ACC.h5'
#checkpointer = ModelCheckpoint(filepath,monitor='val_segmentation_output_jaccard',save_weights_only=True, verbose=1, save_best_only=True)
#checkpointer2 = ModelCheckpoint(filepath1,monitor='val_classification_output_accuracy',save_weights_only=True, verbose=1, save_best_only=True)
filepath="Weights/ClassSegm/Final/14/epoca-{epoch:02d}.h5"
checkpointer2 = ModelCheckpoint(filepath,save_weights_only=True, verbose=1)
earlystopper = EarlyStopping(patience=60,monitor='val_classification_output_accuracy', verbose=1)
log_dire = os.path.join(
    "logs",
    "ClassandSegm","Final","14"
)
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dire, histogram_freq=1)


## Actual Training

In [ ]:
 results=model.fit_generator(generator=training_generator,
                    validation_data=validation_generator,
                    use_multiprocessing=True,
                    steps_per_epoch=2000/(2*batch),
                    workers=6,
                    epochs=150,
                    callbacks=[Reduce,tensorboard_callback,earlystopper,checkpointer2],#checkpointer,checkpointer2,
                    verbose=1)


In [ ]:
plot_performanceJunto(results)

## Model Performance

#### Import Data

In [2]:
DATAPATH = 'ISIC2017/ISIC-2017_Validation_Data/ISIC-2017_Validation_Data_Resize_5721/'
DATAPATH_segm = 'ISIC2017/ISIC-2017_Validation_Part1_GroundTruth/ISIC-2017_Validation_Part1_GroundTruth_Resize_5721/'
DATAPATH_csv='ISIC2017/ISIC-2017_Validation_Part3_GroundTruth.csv'
x_validation,y_validation,y_validationclass,nomefichv=import_datasetBSrot(DATAPATH,DATAPATH_segm, DATAPATH_csv,150)
x_validation=x_validation/255

yes


#### Create Model and import Weights

In [3]:
model = redeUNETconjunta(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS, act='relu',complexity=32,fully=1024,pool=3)

#### Predict Model

In [ ]:
#model.load_weights("Weights\Segmentation\Segmentation7_32_2.h5", by_name=model.load_weights(filepath1)
#model.load_weights(filepath1)Weights/ClassSegm/Final/1/epoca-68.h5
#Weights/ClassSegm/Final/1/epoca-68.h5
model.load_weights("Weights/Class_Segm_CAnovo.h5")
predictions_val = model.predict(x_validation,batch_size=2)

### Example with image and classification

In [ ]:
ix = random.randint(0, 150)
ix=ix-1
ix=90
#Melanoma incorrecto: 106, 127,128,90,76,75,51,25
print(ix)
print("Image:", nomefichv[ix])
print("Prediction:")
print(predictions_val[0][ix])
print(np.argmax(predictions_val[0][ix], axis=0))
print("Groundtruth:")
print(np.argmax(y_validationclass[ix], axis=0))

imshow(x_validation[ix])
plt.show()
preds_val_t = (predictions_val[1] > 0.5).astype(np.uint8)
preds_val_t2 = np.reshape(preds_val_t, (len(preds_val_t),388, 388))
imshow(np.squeeze(preds_val_t2[ix]))
plt.show()
imshow(np.squeeze(y_validation[ix]))

## Calculate Metrics

#### Classification Metrics:

In [ ]:
classification_metrics(predictions_val[0],y_validationclass)

#### Segmentation Metrics:

In [ ]:
preds_val_t = (predictions_val[1] > 0.5).astype(np.uint8)
preds_val_t2 = np.reshape(preds_val_t, (len(preds_val_t),388, 388))
DATAPATH_OUT = 'ISIC2017/ISIC-2017_Validation_Part1_GroundTruth/Result/teste/1/'
automatePostProcess(preds_val_t2,DATAPATH_OUT,nomefichv) 

## Test dataset

In [ ]:
DATAPATH = 'ISIC2017/ISIC-2017_Test_v2_Data/ISIC-2017_Test_v2_Data_Resize_5721/'
DATAPATH_segm = 'ISIC2017/ISIC-2017_Test_v2_Part1_GroundTruth/ISIC-2017_Test_v2_Part1_GroundTruth_Resize_5721/'
DATAPATH_csv='ISIC2017/ISIC-2017_Test_v2_Part3_GroundTruth.csv'
x_validation,y_validation,y_validationclass,nomefichv=import_datasetBSrot(DATAPATH,DATAPATH_segm, DATAPATH_csv,600)
x_validation=x_validation/255

In [ ]:
model.load_weights("Weights/epoca-32.h5")
#model.load_weights("Weights/ClassSegm/Class_Segm_CAnovo.h5")
predictions_val = model.predict(x_validation,batch_size=4)
classification_metrics(predictions_val[0],y_validationclass)
preds_val_t = (predictions_val[1] > 0.5).astype(np.uint8)
preds_val_t2 = np.reshape(preds_val_t, (len(preds_val_t),388, 388))
DATAPATH_OUT = 'ISIC2017/ISIC-2017_Test_v2_Part1_GroundTruth/Result/teste/1/'
automatePostProcessTest(preds_val_t2,DATAPATH_OUT,nomefichv)

In [ ]:
#model.load_weights("Weights/epoca-32.h5")
model.load_weights("Weights/ClassSegm/Class_Segm_CAnovo.h5")
predictions_val = model.predict(x_validation,batch_size=4)
classification_metrics(predictions_val[0],y_validationclass)